# Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns 

### Contexto

No Airbnb, qualquer pessoa que tenha um quarto ou um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles (e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio, desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes extras, exigência de verificação de identidade do locador, etc.

### Nosso objetivo

Construir um modelo de previsão de preço que permita uma pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesmas características) ou não.

### O que temos disponível, inspirações e créditos

As bases de dados foram retiradas do site kaggle: https://www.kaggle.com/allanbruno/airbnb-rio-de-janeiro

Caso queira uma outra solução, podemos olhar como referência a solução do usuário Allan Bruno do kaggle no Notebook: https://www.kaggle.com/allanbruno/helping-regular-people-price-listings-on-airbnb

Você vai perceber semelhanças entre a solução que vamos desenvolver aqui e a dele, mas também algumas diferenças significativas no processo de construção do projeto.

- As bases de dados são os preços dos imóveis obtidos e suas respectivas características em cada mês.
- Os preços são dados em reais (R$)
- Temos bases de abril de 2018 a maio de 2020, com exceção de junho de 2018 que não possui base de dados

### Expectativas Iniciais

- Acredito que a sazonalidade pode ser um fator importante, visto que meses como dezembro costumam ser bem caros no RJ
- A localização do imóvel deve fazer muita diferença no preço, já que no Rio de Janeiro a localização pode mudar completamente as características do lugar (segurança, beleza natural, pontos turísticos)
- Adicionais/Comodidades podem ter um impacto significativo, visto que temos muitos prédios e casas antigos no Rio de Janeiro

Vamos descobrir o quanto esses fatores impactam e se temos outros fatores não tão intuitivos que são extremamente importantes.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import pathlib

### Consolidar Base de Dados

In [ ]:
#definindo o caminho base para os meus arquivos
caminho = pathlib.Path('dataset')

meses = {'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4, 'mai': 5, 'jun': 6, 'jul': 7, 'ago': 8, 'set': 9, 'out': 10, 'nov': 11, 'dez': 12}

airbnb_df = pd.DataFrame()
bases = []

#percorrendo todos os meus arquivos
for arquivo in caminho.iterdir():
    #armazenando o mês do arquivo para colocar na base de dados depois, pois iremos perder essa informação
    nome_mes = arquivo.name[:3]
    mes = meses[nome_mes]
    
    #armazenando o ano do arquivo para colocar na base de dados depois, pois iremos perder essa informação
    ano = arquivo.name[-8:]
    ano = int(ano.replace('.csv', ''))
    base = pd.read_csv(caminho / arquivo.name)
    base['year'] = ano
    base['month'] = mes
    bases.append(base)

airbnb_df = pd.concat(bases)
display(airbnb_df)

 Vou excluir algumas colunas que não vão ajudar no nosso modelo de previsão, para deixar o modelo mais leve e com menos falhas. Para isso vou gerar um arquivo em excel com os mil primeiros dados da tabela para fazer uma análise qualitativa.

In [ ]:
airbnb_df.head(1000).to_csv('registros.csv')